In [7]:
import numpy as np
import cv2 as cv

In [11]:
def flow_to_hsv(flow, mag_clip=None):
    # flow: HxWx2 float32
    fx, fy = flow[..., 0], flow[..., 1]
    mag, ang = cv.cartToPolar(fx, fy, angleInDegrees=False)

    if mag_clip is not None:
        mag = np.clip(mag, 0, mag_clip)

    hsv = np.zeros((flow.shape[0], flow.shape[1], 3), dtype=np.uint8)
    hsv[..., 0] = (ang * 180 / np.pi / 2).astype(np.uint8)  # [0,180)
    hsv[..., 1] = 255
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX).astype(np.uint8)

    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    return bgr, mag

In [3]:
def draw_flow_arrows(img_bgr, flow, step=16, scale=1.0):
    h, w = img_bgr.shape[:2]
    out = img_bgr.copy()

    y, x = np.mgrid[step//2:h:step, step//2:w:step].astype(int)
    fx = flow[y, x, 0]
    fy = flow[y, x, 1]

    for (x0, y0, dx, dy) in zip(x.ravel(), y.ravel(), fx.ravel(), fy.ravel()):
        x1 = int(x0 + scale * dx)
        y1 = int(y0 + scale * dy)
        cv.arrowedLine(out, (x0, y0), (x1, y1), (0, 255, 0), 1, tipLength=0.3)

    return out

**Farnebäck parameters to tune**:

winsize (window size):

- Larger → smoother, more robust to noise, worse for thin motion boundaries, slower.

- Smaller → better detail, more fragile/noisy.

levels (pyramid levels):

- More levels → can handle larger motion (but slower).

- Too few → fast objects “break” or smear.

iterations:

- More → better convergence, slower.

poly_n / poly_sigma (polynomial neighborhood and smoothing):

- Larger poly_n + larger poly_sigma → smoother flow, less detail, more robust.

**Rule of thumb**:

- If motion is fast → increase levels, maybe winsize.

- If flow is too blobby → decrease winsize or poly_sigma.

- If flow is sparkly/noisy → increase winsize or poly_sigma, or downscale more.

In [22]:
def main():
    video_path = input("Enter path to video file: ").strip().strip('"')
    cap = cv.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {video_path}")

    ok, frame0 = cap.read()
    if not ok:
        raise RuntimeError("Cannot read first frame.")

    # Real-time-ish speed: shrink frames.
    resize_scale = 0.5  # try 0.5 for more FPS
    frame0 = cv.resize(frame0, None, fx=resize_scale, fy=resize_scale, interpolation=cv.INTER_AREA)
    prev_gray = cv.cvtColor(frame0, cv.COLOR_BGR2GRAY)

    # Farneback parameters (balanced defaults)
    pyr_scale = 0.5
    levels = 4
    winsize = 15
    iterations = 3
    poly_n = 5
    poly_sigma = 1.2
    flags = 0

    arrow_step = 18
    arrow_scale = 1.5

    # Motion mask threshold (in pixels/frame after resizing)
    mag_thresh = 1.5

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        frame = cv.resize(frame, None, fx=resize_scale, fy=resize_scale, interpolation=cv.INTER_AREA)
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        flow = cv.calcOpticalFlowFarneback(
            prev_gray, gray, None,
            pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags
        ).astype(np.float32)

        hsv_bgr, mag = flow_to_hsv(flow, mag_clip=None)

        arrows = draw_flow_arrows(frame, flow, step=arrow_step, scale=arrow_scale)

        motion_mask = (mag > mag_thresh).astype(np.uint8) * 255
        motion_mask = cv.medianBlur(motion_mask, 5)

        motion_overlay = frame.copy()
        motion_overlay[motion_mask > 0] = (0.5 * motion_overlay[motion_mask > 0] + 0.5 * np.array([0, 0, 255])).astype(np.uint8)

        cv.imshow("Frame (arrows)", arrows)
        cv.imshow("Dense flow HSV", hsv_bgr)
        cv.imshow("Motion mask overlay", motion_overlay)

        key = cv.waitKey(1) & 0xFF
        if key == 27:  # ESC
            break
        elif key == ord('['):
            mag_thresh = max(0.0, mag_thresh - 0.25)
            print("mag_thresh:", mag_thresh)
        elif key == ord(']'):
            mag_thresh += 0.25
            print("mag_thresh:", mag_thresh)

        prev_gray = gray

    cap.release()
    cv.destroyAllWindows()

C:\Users\BlackySwanny\Downloads\video_2025-12-16_03-45-11.mp4

In [23]:
main()

Enter path to video file:         C:\Users\BlackySwanny\Downloads\video_2025-12-16_03-45-11.mp4
